## Loading and training on new dataset

In [18]:
from torch.utils.data import Dataset,DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transform
from torchvision.transforms import ToTensor
import torchvision.datasets as datasets
from torch.utils.data import Dataset,DataLoader

class CNNClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Conv2d(1, 64, 3),
                                 nn.ReLU(),
                                 nn.MaxPool2d((2, 2), stride=2),
                                 nn.Conv2d(64, 128, 3),
                                 nn.ReLU(),
                                 nn.MaxPool2d((2, 2), stride=2),
                                 nn.Conv2d(128, 64, 3),
                                 nn.ReLU(),
                                 nn.MaxPool2d((2, 2), stride=2),
                                 )
        self.classification_head = nn.Sequential(nn.Linear(64, 20, bias=True),
                                                 nn.ReLU(),
                                                 nn.Linear(20, 10, bias=True), )

    def forward(self, x):
        features = self.net(x)
        return self.classification_head(features.view(batch_size, -1))


batch_size=4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNClassifier().to(device)
check_point = torch.load("./checkpoint.pt")
model.load_state_dict(check_point["model_state_dict"])
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=0.001)
optimizer.load_state_dict(check_point["optimizer_state_dict"])
loss = check_point["loss"]
epoch = check_point["epoch"]

num_ftrs = model.classification_head[2].in_features
model.classification_head[2] = nn.Linear(num_ftrs,2)
model.to(device)
print(model)

def generateGaussian(tindx):
    return torch.normal(tindx[0], tindx[1], (1, 28, 28))
# Define the custom dataset class
class MyDataset(Dataset):
    def __init__(self, n):
        classes = {0: (0.5, 2), 1: (1, 2.5)}
        self.y = [torch.round(torch.rand(1))[0].long() for i in range(n)]
        self.X = [generateGaussian(classes[self.y[i].item()]) for i in range(n)]

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

    def __len__(self):
        return len(self.X)
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


dataset = MyDataset(1000)
dataloader = DataLoader(dataset,batch_size=batch_size,shuffle=True)
test_dataset = MyDataset(200)
testloader = DataLoader(test_dataset,batch_size=batch_size,shuffle=True)

NEW_EPOCHS = 5
for epoch in range(epoch,NEW_EPOCHS+1):
    running_loss = loss
    for i, data in enumerate(dataloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        _,pred_outputs = torch.max(outputs,1)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 50 == 25:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 25))
            running_loss = 0.0

print(f"Loss = {loss.item()}")

correct_train, total_train = 0,0
for i, data in enumerate(dataloader):
    inputs, labels = data[0].to(device), data[1].to(device)
    outputs = model(inputs)
    _, predicted = torch.max(outputs, 1)
    total_train += labels.size(0)
    correct_train += (predicted == labels).sum()
               
print("For Training: ") 
print(f"Correct = {correct_train}, Total = {total_train}")           
print(f"Loss = {loss.item()}, Training Accuracy = {correct_train/total_train*100}")

correct, total = 0, 0
for i, vdata in enumerate(testloader):
    tinputs, tlabels = vdata[0].to(device), vdata[1].to(device)
    toutputs = model(tinputs)
    _, predicted = torch.max(toutputs, 1)
    total += tlabels.size(0)
    correct += (predicted == tlabels).sum()
    
print("For Testing:")
print(f"Correct = {correct}, Total = {total}")
accuracy = 100.0 * correct / total
print("The overall testing accuracy is {}".format(accuracy))

CNNClassifier(
  (net): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classification_head): Sequential(
    (0): Linear(in_features=64, out_features=20, bias=True)
    (1): ReLU()
    (2): Linear(in_features=20, out_features=2, bias=True)
  )
)
[3,    26] loss: 1.785
[3,    76] loss: 1.433
[3,   126] loss: 1.318
[3,   176] loss: 1.182
[3,   226] loss: 1.206
[4,    26] loss: 0.546
[4,    76] loss: 1.015
[4,   126] loss: 0.996
[4,   176] loss: 0.543
[4,   226] loss: 0.722
[5,    26] loss: 0.569
[5,    76] loss: 0.672
[5,   126] lo